In [46]:
# %%capture
# !pip install pyserini==0.10.0.1

import os
import numpy as np
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

In [2]:
from pyserini.search import get_topics

topics = get_topics('msmarco_passage_dev_subset')
print(f'{len(topics)} queries total')

6980 queries total


In [3]:
topics[1102400]['title']

'why do bears hibernate'

In [4]:
from pyserini.search import SimpleSearcher

searcher = SimpleSearcher.from_prebuilt_index('/home/shivani/Downloads/index-cast2019.tar.gz/index-cast2019')

Attempting to initialize pre-built index /home/shivani/Downloads/index-cast2019.tar.gz/index-cast2019.
Unrecognized index name /home/shivani/Downloads/index-cast2019.tar.gz/index-cast2019
Initializing /home/shivani/Downloads/index-cast2019.tar.gz/index-cast2019...


In [5]:
SimpleSearcher.from_prebuilt_index()

TypeError: from_prebuilt_index() missing 1 required positional argument: 'prebuilt_index_name'

In [6]:
import json

hits = searcher.search('why do we fall in love?')

# Prints the first 10 hits/
for i in range(0, 10):
    jsondoc = hits[i].raw
    print(f'{i+1:2} {hits[i].score:.5f} {jsondoc[:80]}..., {hits[i].docid}')

 1 14.58490 The #1 reason why people fall out of love is because they're human. Yes. We are ..., MARCO_3862237
 2 13.60040 Of course, there are numerous reasons why people break up, but they are not alwa..., MARCO_3862236
 3 13.50850 The #1 reason why people fall out of love is because they're human. Yes. We are ..., MARCO_1828466
 4 12.92740 Conclusion. The knowledge of the 3 reasons why people can fall out of love can b..., MARCO_3862239
 5 12.86610 "I Love My Life" was written by Lukas Bellesini, Jayson DeZuzio, Wolfgang Gartne..., CAR_a9cc9dbb4a421ab9e3f2e63526698de267f30ffc
 6 12.77540 Two soundtrack albums were released for Why Do Fools Fall in Love by Warner Musi..., MARCO_7974192
 7 12.69900 Why do we love? Why do we hate. Why, why, why? Well, these are the questions and..., MARCO_7903322
 8 12.68660 Lymon's music and story were re-introduced to modern audiences with Why Do Fools..., CAR_31c7c8152961f9d06f9a0c7f947fbb56948f04d8
 9 12.50680 You always hurt the one you love, the 

In [7]:
hits[i].raw

'Also in 1996 Thompson performed background vocals for the song "Cold Rock A Party" by MC Lyte featuring Missy Elliott and Puff Daddy. She also appeared on Lyte\'s 1998 album Seven & Seven, on the track \'It\'s All Yours". Later work included "Why Do Fools Fall in Love", a re-imagining of the 1955 Frankie Lymon & the Teenagers hit of the same name. Thompson\'s "Why Do Fools Fall in Love," featuring Missy Elliott and rapper Mocha, was featured in the 1998 Lymon biopic Why Do Fools Fall in Love and its soundtrack. The singer also appeared on Missy Elliott\'s second album, Da Real World. By the latter part of 1998, Thompson\'s debut was certified Platinum by the RIAA after an estimate of one million copies of the album were sold.'

In [63]:
import json
import string 
with open('./treccastweb/2020/2020_manual_evaluation_topics_v1.0.json') as f:
    loaded_json = json.load(f)

In [108]:

r_s, r_q = 1, 1
theta = 17
# s.translate(str.maketrans('', '', string.punctuation))/
for conv in loaded_json[1:2]:
    w_s, w_q = [], []
    print ("Processing for {}".format(conv['number']))
    for idx, turn in enumerate(conv['turn']):
        query = turn['manual_rewritten_utterance']
        query = query.translate(str.maketrans('', '', string.punctuation))
        w_q.append([])
        print ("Turn {} -> {}".format(turn['number'], turn['manual_rewritten_utterance']))
        for word in query.split():
            word = word.translate(str.maketrans('', '', string.punctuation))
            text = searcher.search(word)
            if len(text) > 0:
                match_score = text[0].score
                if match_score > r_q:
                    w_q[idx].append(word)
                if match_score > r_s:
                    w_s.append(word)
        if idx>0:
            query_res = searcher.search(query)
            query += ' '.join(W_s)
            if query_res[0].score < theta:
                if idx < 3:
                    range_ = np.arange(0, idx)
                else:
                    range_ = np.arange(idx-3, idx)

                for item in range_:
                    query += ' ' + ' '.join(w_q[item])
        query = ' '.join(set(query.split()))
        print("\n{}\ndoc id is {}".format(query,searcher.search(query)[0].docid))               
                    

Processing for 82
Turn 1 -> I would like to learn about GMO Food labeling.

learn like GMO labeling I about to Food would
doc id is MARCO_1209363
Turn 2 -> What are the pros and cons of GMO food labeling?

pros the and GMO of labeling food What are cons
doc id is MARCO_3311529
Turn 3 -> What are the cons of GMO food labeling?

the GMO of labeling food What are cons
doc id is MARCO_3311529
Turn 4 -> What are the GMO food labeling rules in the EU?

rules the GMO labeling food What in EU are
doc id is CAR_14e62ec6fa1fb134487c7a2ce2a490b937880bb0
Turn 5 -> Tell me about traceability tools for GMO foods in the EU.

the me tools GMO about traceability in for Tell foods EU
doc id is CAR_ef1121e0a382dca7bffdf33a03ed9e9657fe65f4
Turn 6 -> What is the role of Co-Extra in GMO food traceability in the EU?

role is the GMO of food What in traceability CoExtra EU
doc id is CAR_ad707e094a5475b5b0a35ab384901cb37e750c7d
Turn 7 -> How is food tested for GMO contamination?

role is rules contamination me

In [87]:
w_s, w_q

(['garage',
  'garage',
  'garage',
  'garage',
  'garage',
  'smart',
  'garage',
  'smart',
  'garage',
  'smart',
  'garage',
  'hacked'],
 [['know', 'garage', 'door', 'bad'],
  ['my', 'garage', 'door', 'stopped', 'Why'],
  ['cost', 'someone', 'repair', 'garage', 'door'],
  ['cost', 'garage', 'door'],
  ['choose', 'garage', 'door'],
  ['smart', 'garage', 'door'],
  ['me', 'know', 'safety', 'smart', 'garage', 'door'],
  ['smart', 'garage', 'door', 'hacked']])